In [1]:
import matplotlib.pyplot as plt
import numpy as np
import copy
import h5py
import os
import seaborn as sns
from pyscf.scf.uhf import det_ovlp

In [2]:
# Wavefunction generation
from pyscf import lib, gto, scf
from pyscf import gto, mp, mcscf
def H2_casci(scf_checkfile, ci_checkfile):
    mol = gto.M(
        atom="H 0. 0. 0.0; H 0. 0. 1.4",
        basis=f"ccecpccpvqz",
        unit="bohr",
        charge=0,
        spin=0,
        verbose=1,
    )
    mf = scf.UHF(mol).run()
    mf.chkfile = scf_checkfile
    mf.kernel()
    mc = mcscf.CASCI(mf, 3,2)
    # mc.fcisolver.nroots = 4
    mc.kernel()
    print(mc.__dict__.keys())
    with h5py.File(ci_checkfile, "a") as f:
        f.create_group("ci")
        f["ci/ncas"] = mc.ncas
        f["ci/nelecas"] = list(mc.nelecas)
        f["ci/ci"] = mc.ci
        f["ci/mo_coeff"] = mc.mo_coeff    
    return mol, mf, mc

In [3]:
scf_checkfile = 'rohf.chk'
ci_checkfile = 'casci.chk'
for fname in [scf_checkfile, ci_checkfile]:
    if os.path.isfile(fname):
        os.remove(fname)
mol, mf, mcc = H2_casci(scf_checkfile, ci_checkfile)

dict_keys(['mol', '_scf', 'verbose', 'stdout', 'max_memory', 'ncas', 'nelecas', '_ncore', 'fcisolver', 'frozen', 'extrasym', 'e_tot', 'e_cas', 'ci', 'mo_coeff', 'mo_energy', 'mo_occ', 'converged'])


In [4]:
import bosonslater 
import pyscftools, mc
wf1 = bosonslater.BosonWF(mol, mf, mc=mcc)

nconfig = 1000
mol, mf = pyscftools.recover_pyscf(scf_checkfile)
configs = mc.initial_guess(mol, nconfig)

In [ ]:
signb, phib = wf1.recompute(configs)
sign, logval = wf1.value()
val = np.exp(logval)
rho = val**2 
rho.shape

In [ ]:
inv_center = np.array([0., 0., 0.7])

In [26]:
from wftools import generate_slater
wfs = []
num_det = mcc.ci.shape[0] * mcc.ci.shape[1]

for i in range(mcc.ci.shape[0]):
    for j in range(mcc.ci.shape[1]):
        mc0 = copy.copy(mcc)
        mc0.ci = mcc.ci * 0
        mc0.ci[i, j] = 1 # zero all coefficients except one, this is probably quite inefficient, but should work for now.
        wf0, _ = generate_slater(mol, mf, mc=mc0, optimize_determinants=False)
        wfs.append(wf0)

In [60]:
configs.configs[0]

array([[-0.16270017, -0.96467596, -1.70554183],
       [-0.95214724,  1.39128028,  2.15723709]])

# Inversion

In [61]:
conf2 = copy.deepcopy(configs)
conf2.configs = 2*inv_center - conf2.configs
conf2.configs[0]

array([[ 0.16270017,  0.96467596,  3.10554183],
       [ 0.95214724, -1.39128028, -0.75723709]])

In [109]:
configs.configs[0]

array([[-0.16270017, -0.96467596, -1.70554183],
       [-0.95214724,  1.39128028,  2.15723709]])

In [57]:
for i, wf in enumerate(wfs): 
    s0, lv0 = wf.recompute(configs)
    s1, lv1 = wf.recompute(conf2)
    # print(np.sum(s0), np.sum(s1))
    v0 = s0 * np.exp(lv0)
    v1 = s1 * np.exp(lv1)
    if (np.abs(v1 - v0) < 10**-12).all():
        print(i, '1')
    elif (np.abs(v1 + v0) < 10**-12).all():
        print(i, '-1')
        
    # prod = s1 * np.exp(

0 1
1 -1
2 1
3 -1
4 1
5 -1
6 1
7 -1
8 1


# Reflection on YZ plane

In [106]:
conf2 = copy.deepcopy(configs)
# conf2.configs = conf2.configs * np.array([-1, 1, 1])
conf2.configs[:, :, 0]= 2*inv_center[0] - conf2.configs[:, :, 0]
conf2.configs[0]

array([[ 0.16270017, -0.96467596, -1.70554183],
       [ 0.95214724,  1.39128028,  2.15723709]])

In [107]:
for i, wf in enumerate(wfs): 
    s0, lv0 = wf.recompute(configs)
    s1, lv1 = wf.recompute(conf2)
    # print(np.sum(s0), np.sum(s1))
    v0 = s0 * np.exp(lv0)
    v1 = s1 * np.exp(lv1)
    if (np.abs(v1 - v0) < 10**-12).all():
        print(i, '1')
    elif (np.abs(v1 + v0) < 10**-12).all():
        print(i, '-1')
        
    # prod = s1 * np.exp(

0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1


# Reflection on XZ plane

In [104]:
conf2 = copy.deepcopy(configs)
# conf2.configs = conf2.configs * np.array([1, -1, 1])
conf2.configs[:, :, 1]= 2*inv_center[1] - conf2.configs[:, :, 1]
conf2.configs[0]

array([[-0.16270017,  0.96467596, -1.70554183],
       [-0.95214724, -1.39128028,  2.15723709]])

In [108]:
configs.configs[0]

array([[-0.16270017, -0.96467596, -1.70554183],
       [-0.95214724,  1.39128028,  2.15723709]])

In [105]:
for i, wf in enumerate(wfs): 
    s0, lv0 = wf.recompute(configs)
    s1, lv1 = wf.recompute(conf2)
    # print(np.sum(s0), np.sum(s1))
    v0 = s0 * np.exp(lv0)
    v1 = s1 * np.exp(lv1)
    if (np.abs(v1 - v0) < 10**-12).all():
        print(i, '1')
    elif (np.abs(v1 + v0) < 10**-12).all():
        print(i, '-1')
        
    # prod = s1 * np.exp(

0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1


# Reflection on XY plane

In [99]:
conf2 = copy.deepcopy(configs)
conf2.configs[:, :, 2]= 2*inv_center[2] - conf2.configs[:, :, 2]
conf2.configs[0]

array([[-0.16270017, -0.96467596,  3.10554183],
       [-0.95214724,  1.39128028, -0.75723709]])

In [100]:
configs.configs[0]

array([[-0.16270017, -0.96467596, -1.70554183],
       [-0.95214724,  1.39128028,  2.15723709]])

In [103]:
for i, wf in enumerate(wfs): 
    s0, lv0 = wf.recompute(configs)
    s1, lv1 = wf.recompute(conf2)
    # print(np.sum(s0), np.sum(s1))
    v0 = s0 * np.exp(lv0)
    v1 = s1 * np.exp(lv1)
    if (np.abs(v1 - v0) < 10**-12).all():
        print(i, '1')
    elif (np.abs(v1 + v0) < 10**-12).all():
        print(i, '-1')
        
    # prod = s1 * np.exp(

0 1
1 -1
2 1
3 -1
4 1
5 -1
6 1
7 -1
8 1
